# Distribution Analysis for scGPT Fine-tuning: Training vs Testing Data

This notebook analyzes the distribution differences between training and testing data in the scGPT fine-tuning pipeline to investigate potential out-of-distribution (OOD) issues that might affect model generalization.

## Overview
- **Goal**: Verify if fine-tuning performance issues are due to distribution shifts between train/test data
- **Dataset**: Adamson perturbation data with simulation split
- **Analysis**: Statistical and visual comparison of gene expression patterns, perturbation effects, and cell characteristics


In [ ]:
# Setup and Installation for Google Colab
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/GitHub/Biological-Foundation-Model/notebooks/scGPT_finetune')

# Install required packages
%pip install -r ./requirements.txt
%pip install scgpt "flash-attn<1.0.5"
%pip install seaborn plotly scipy


In [ ]:
# Import libraries
import json
import os
import sys
import time
import copy
from pathlib import Path
from typing import Iterable, List, Tuple, Dict, Union, Optional
import warnings

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
from scipy.spatial.distance import pdist, squareform
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import pairwise_distances
from torch import nn
from torch.nn import functional as F
from torchtext.vocab import Vocab
from torchtext._torchtext import (
    Vocab as VocabPybind,
)
from torch_geometric.loader import DataLoader
from gears import PertData, GEARS
from gears.inference import compute_metrics, deeper_analysis, non_dropout_analysis
from gears.utils import create_cell_graph_dataset_for_prediction

sys.path.insert(0, "../")

import scgpt as scg
from scgpt.model import TransformerGenerator
from scgpt.loss import (
    masked_mse_loss,
    criterion_neg_log_bernoulli,
    masked_relative_error,
)
from scgpt.tokenizer import tokenize_batch, pad_batch, tokenize_and_pad_batch
from scgpt.tokenizer.gene_tokenizer import GeneVocab
from scgpt.utils import set_seed, map_raw_id_to_vocab_id, compute_perturbation_metrics

# Set up plotting
plt.style.use('default')
sns.set_palette("husl")
matplotlib.rcParams["savefig.transparent"] = False
warnings.filterwarnings("ignore")

set_seed(42)
print("Libraries imported successfully!")


In [ ]:
# Load and prepare data
print("Loading perturbation data...")

# Settings for data processing
pad_token = "<pad>"
special_tokens = [pad_token, "<cls>", "<eoc>"]
pad_value = 0
pert_pad_id = 0
include_zero_gene = "all"
max_seq_len = 1536

# Dataset settings
data_name = "adamson"
split = "simulation"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load perturbation data
pert_data = PertData("./data")
pert_data.load(data_name=data_name)
pert_data.prepare_split(split=split, seed=1)
pert_data.get_dataloader(batch_size=64, test_batch_size=64)

print(f"Data loaded successfully!")
print(f"Dataset: {data_name}")
print(f"Split: {split}")
print(f"Device: {device}")

# Get basic info about the dataset
adata = pert_data.adata
print(f"\nDataset info:")
print(f"Total cells: {adata.n_obs}")
print(f"Total genes: {adata.n_vars}")
print(f"Conditions: {adata.obs['condition'].unique()}")
print(f"Split composition: {pert_data.subgroup}")


In [ ]:
# Extract train/test splits for analysis
def extract_split_data(adata, split_info, split_name):
    """Extract data for a specific split"""
    if split_name == "train":
        split_cells = split_info["train_idx"]
    elif split_name == "test":
        split_cells = split_info["test_idx"]
    elif split_name == "val":
        split_cells = split_info["val_idx"]
    else:
        raise ValueError(f"Unknown split: {split_name}")
    
    return adata[split_cells].copy()

# Get split information
train_adata = extract_split_data(adata, pert_data.split, "train")
test_adata = extract_split_data(adata, pert_data.split, "test")
val_adata = extract_split_data(adata, pert_data.split, "val")

print(f"Split sizes:")
print(f"Train: {train_adata.n_obs} cells")
print(f"Test: {test_adata.n_obs} cells") 
print(f"Val: {val_adata.n_obs} cells")

# Get condition distributions
print(f"\nCondition distributions:")
print(f"Train conditions: {train_adata.obs['condition'].value_counts()}")
print(f"Test conditions: {test_adata.obs['condition'].value_counts()}")
print(f"Val conditions: {val_adata.obs['condition'].value_counts()}")


In [ ]:
# 1. Gene Expression Distribution Analysis
print("=== GENE EXPRESSION DISTRIBUTION ANALYSIS ===")

def analyze_gene_expression_distribution(adata, split_name):
    """Analyze gene expression distribution for a split"""
    # Convert to dense if sparse
    if hasattr(adata.X, 'toarray'):
        X = adata.X.toarray()
    else:
        X = adata.X
    
    # Calculate statistics
    mean_expr = np.mean(X, axis=0)
    std_expr = np.std(X, axis=0)
    var_expr = np.var(X, axis=0)
    
    # Calculate per-cell statistics
    cell_means = np.mean(X, axis=1)
    cell_stds = np.std(X, axis=1)
    cell_zeros = np.sum(X == 0, axis=1) / X.shape[1]  # fraction of zeros per cell
    
    return {
        'gene_means': mean_expr,
        'gene_stds': std_expr,
        'gene_vars': var_expr,
        'cell_means': cell_means,
        'cell_stds': cell_stds,
        'cell_zeros': cell_zeros,
        'total_expression': np.sum(X, axis=1),
        'n_genes_expressed': np.sum(X > 0, axis=1)
    }

# Analyze each split
train_stats = analyze_gene_expression_distribution(train_adata, "train")
test_stats = analyze_gene_expression_distribution(test_adata, "test")
val_stats = analyze_gene_expression_distribution(val_adata, "val")

print(f"Gene expression statistics calculated for all splits")


In [ ]:
# Visualize gene expression distributions
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Gene Expression Distribution Comparison: Train vs Test', fontsize=16)

# 1. Gene mean expression
axes[0, 0].hist(train_stats['gene_means'], bins=50, alpha=0.7, label='Train', density=True)
axes[0, 0].hist(test_stats['gene_means'], bins=50, alpha=0.7, label='Test', density=True)
axes[0, 0].set_xlabel('Mean Gene Expression')
axes[0, 0].set_ylabel('Density')
axes[0, 0].set_title('Gene Mean Expression Distribution')
axes[0, 0].legend()

# 2. Gene expression variance
axes[0, 1].hist(train_stats['gene_vars'], bins=50, alpha=0.7, label='Train', density=True)
axes[0, 1].hist(test_stats['gene_vars'], bins=50, alpha=0.7, label='Test', density=True)
axes[0, 1].set_xlabel('Gene Expression Variance')
axes[0, 1].set_ylabel('Density')
axes[0, 1].set_title('Gene Expression Variance Distribution')
axes[0, 1].legend()

# 3. Cell mean expression
axes[0, 2].hist(train_stats['cell_means'], bins=50, alpha=0.7, label='Train', density=True)
axes[0, 2].hist(test_stats['cell_means'], bins=50, alpha=0.7, label='Test', density=True)
axes[0, 2].set_xlabel('Mean Expression per Cell')
axes[0, 2].set_ylabel('Density')
axes[0, 2].set_title('Cell Mean Expression Distribution')
axes[0, 2].legend()

# 4. Fraction of zeros per cell
axes[1, 0].hist(train_stats['cell_zeros'], bins=50, alpha=0.7, label='Train', density=True)
axes[1, 0].hist(test_stats['cell_zeros'], bins=50, alpha=0.7, label='Test', density=True)
axes[1, 0].set_xlabel('Fraction of Zeros per Cell')
axes[1, 0].set_ylabel('Density')
axes[1, 0].set_title('Sparsity Distribution')
axes[1, 0].legend()

# 5. Number of genes expressed per cell
axes[1, 1].hist(train_stats['n_genes_expressed'], bins=50, alpha=0.7, label='Train', density=True)
axes[1, 1].hist(test_stats['n_genes_expressed'], bins=50, alpha=0.7, label='Test', density=True)
axes[1, 1].set_xlabel('Number of Genes Expressed')
axes[1, 1].set_ylabel('Density')
axes[1, 1].set_title('Genes Expressed per Cell')
axes[1, 1].legend()

# 6. Total expression per cell
axes[1, 2].hist(train_stats['total_expression'], bins=50, alpha=0.7, label='Train', density=True)
axes[1, 2].hist(test_stats['total_expression'], bins=50, alpha=0.7, label='Test', density=True)
axes[1, 2].set_xlabel('Total Expression per Cell')
axes[1, 2].set_ylabel('Density')
axes[1, 2].set_title('Total Expression Distribution')
axes[1, 2].legend()

plt.tight_layout()
plt.show()


In [ ]:
# Statistical tests for distribution differences
print("=== STATISTICAL TESTS FOR DISTRIBUTION DIFFERENCES ===")

def perform_statistical_tests(train_stats, test_stats, metric_name):
    """Perform statistical tests comparing train and test distributions"""
    train_data = train_stats[metric_name]
    test_data = test_stats[metric_name]
    
    # Kolmogorov-Smirnov test
    ks_stat, ks_pvalue = stats.ks_2samp(train_data, test_data)
    
    # Mann-Whitney U test (non-parametric)
    mw_stat, mw_pvalue = stats.mannwhitneyu(train_data, test_data, alternative='two-sided')
    
    # Effect size (Cohen's d)
    pooled_std = np.sqrt(((len(train_data) - 1) * np.var(train_data) + 
                         (len(test_data) - 1) * np.var(test_data)) / 
                        (len(train_data) + len(test_data) - 2))
    cohens_d = (np.mean(train_data) - np.mean(test_data)) / pooled_std
    
    return {
        'ks_stat': ks_stat,
        'ks_pvalue': ks_pvalue,
        'mw_stat': mw_stat,
        'mw_pvalue': mw_pvalue,
        'cohens_d': cohens_d,
        'train_mean': np.mean(train_data),
        'test_mean': np.mean(test_data),
        'train_std': np.std(train_data),
        'test_std': np.std(test_data)
    }

# Test different metrics
metrics_to_test = ['gene_means', 'gene_vars', 'cell_means', 'cell_zeros', 
                   'n_genes_expressed', 'total_expression']

statistical_results = {}
for metric in metrics_to_test:
    print(f"\nTesting {metric}:")
    results = perform_statistical_tests(train_stats, test_stats, metric)
    statistical_results[metric] = results
    
    print(f"  Train mean: {results['train_mean']:.4f} ± {results['train_std']:.4f}")
    print(f"  Test mean:  {results['test_mean']:.4f} ± {results['test_std']:.4f}")
    print(f"  Cohen's d:  {results['cohens_d']:.4f}")
    print(f"  KS test:    stat={results['ks_stat']:.4f}, p={results['ks_pvalue']:.4f}")
    print(f"  MW test:    stat={results['mw_stat']:.4f}, p={results['mw_pvalue']:.4f}")
    
    # Interpretation
    if results['ks_pvalue'] < 0.05:
        print(f"  *** SIGNIFICANT DIFFERENCE (KS test, p < 0.05) ***")
    if abs(results['cohens_d']) > 0.2:
        effect_size = "small" if abs(results['cohens_d']) < 0.5 else "medium" if abs(results['cohens_d']) < 0.8 else "large"
        print(f"  *** {effect_size.upper()} EFFECT SIZE (Cohen's d = {results['cohens_d']:.3f}) ***")


In [ ]:
# 2. Perturbation Pattern Analysis
print("\n=== PERTURBATION PATTERN ANALYSIS ===")

def analyze_perturbation_patterns(train_adata, test_adata):
    """Analyze perturbation patterns between train and test sets"""
    
    # Get perturbation conditions (exclude control)
    train_conditions = train_adata.obs['condition'].value_counts()
    test_conditions = test_adata.obs['condition'].value_counts()
    
    # Get unique perturbations
    train_perts = set(train_conditions.index) - {'ctrl'}
    test_perts = set(test_conditions.index) - {'ctrl'}
    
    # Analyze perturbation overlap
    common_perts = train_perts.intersection(test_perts)
    train_only = train_perts - test_perts
    test_only = test_perts - train_perts
    
    # Analyze perturbation types
    def categorize_perturbation(pert_name):
        if '+' in pert_name:
            return 'combinatorial'
        else:
            return 'single'
    
    train_pert_types = [categorize_perturbation(p) for p in train_perts]
    test_pert_types = [categorize_perturbation(p) for p in test_perts]
    
    return {
        'train_conditions': train_conditions,
        'test_conditions': test_conditions,
        'common_perts': common_perts,
        'train_only': train_only,
        'test_only': test_only,
        'train_pert_types': train_pert_types,
        'test_pert_types': test_pert_types,
        'train_single_count': train_pert_types.count('single'),
        'train_combo_count': train_pert_types.count('combinatorial'),
        'test_single_count': test_pert_types.count('single'),
        'test_combo_count': test_pert_types.count('combinatorial')
    }

pert_analysis = analyze_perturbation_patterns(train_adata, test_adata)

print(f"Perturbation overlap analysis:")
print(f"  Common perturbations: {len(pert_analysis['common_perts'])}")
print(f"  Train-only perturbations: {len(pert_analysis['train_only'])}")
print(f"  Test-only perturbations: {len(pert_analysis['test_only'])}")

print(f"\nPerturbation type distribution:")
print(f"  Train - Single: {pert_analysis['train_single_count']}, Combinatorial: {pert_analysis['train_combo_count']}")
print(f"  Test - Single: {pert_analysis['test_single_count']}, Combinatorial: {pert_analysis['test_combo_count']}")

print(f"\nTest-only perturbations: {list(pert_analysis['test_only'])}")
print(f"Train-only perturbations: {list(pert_analysis['train_only'])}")


In [ ]:
# 3. Dimensionality Reduction Analysis
print("\n=== DIMENSIONALITY REDUCTION ANALYSIS ===")

def perform_dimensionality_reduction(adata, method='PCA', n_components=50):
    """Perform dimensionality reduction on gene expression data"""
    # Convert to dense if sparse
    if hasattr(adata.X, 'toarray'):
        X = adata.X.toarray()
    else:
        X = adata.X
    
    if method == 'PCA':
        reducer = PCA(n_components=n_components, random_state=42)
    elif method == 'tSNE':
        reducer = TSNE(n_components=2, random_state=42, perplexity=30)
    else:
        raise ValueError(f"Unknown method: {method}")
    
    embedding = reducer.fit_transform(X)
    return embedding, reducer

# Perform PCA on each split
print("Computing PCA embeddings...")
train_pca, train_pca_model = perform_dimensionality_reduction(train_adata, 'PCA', 50)
test_pca, test_pca_model = perform_dimensionality_reduction(test_adata, 'PCA', 50)
val_pca, val_pca_model = perform_dimensionality_reduction(val_adata, 'PCA', 50)

# Perform t-SNE on combined data for visualization
print("Computing t-SNE embedding...")
combined_adata = train_adata.concatenate([test_adata, val_adata], 
                                        batch_key='split', 
                                        batch_categories=['train', 'test', 'val'])
combined_tsne, _ = perform_dimensionality_reduction(combined_adata, 'tSNE', 2)

print(f"PCA explained variance ratios (first 10 components):")
print(f"  Train: {train_pca_model.explained_variance_ratio_[:10]}")
print(f"  Test:  {test_pca_model.explained_variance_ratio_[:10]}")
print(f"  Val:   {val_pca_model.explained_variance_ratio_[:10]}")


In [ ]:
# Visualize dimensionality reduction results
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# 1. PCA comparison (first 2 components)
axes[0].scatter(train_pca[:, 0], train_pca[:, 1], alpha=0.6, label='Train', s=20)
axes[0].scatter(test_pca[:, 0], test_pca[:, 1], alpha=0.6, label='Test', s=20)
axes[0].set_xlabel(f'PC1 ({train_pca_model.explained_variance_ratio_[0]:.2%} variance)')
axes[0].set_ylabel(f'PC2 ({train_pca_model.explained_variance_ratio_[1]:.2%} variance)')
axes[0].set_title('PCA: Train vs Test')
axes[0].legend()

# 2. t-SNE visualization
split_colors = ['red' if s == 'train' else 'blue' if s == 'test' else 'green' for s in combined_adata.obs['split']]
axes[1].scatter(combined_tsne[:, 0], combined_tsne[:, 1], c=split_colors, alpha=0.6, s=20)
axes[1].set_xlabel('t-SNE 1')
axes[1].set_ylabel('t-SNE 2')
axes[1].set_title('t-SNE: Train (red) vs Test (blue) vs Val (green)')

# 3. PCA variance explained
axes[2].plot(range(1, 11), train_pca_model.explained_variance_ratio_[:10], 'o-', label='Train')
axes[2].plot(range(1, 11), test_pca_model.explained_variance_ratio_[:10], 's-', label='Test')
axes[2].set_xlabel('Principal Component')
axes[2].set_ylabel('Explained Variance Ratio')
axes[2].set_title('PCA Explained Variance')
axes[2].legend()

plt.tight_layout()
plt.show()


In [ ]:
# 4. Distance-based Analysis
print("\n=== DISTANCE-BASED ANALYSIS ===")

def compute_distances_between_splits(train_adata, test_adata, n_samples=1000):
    """Compute distances between train and test splits"""
    
    # Sample cells for computational efficiency
    if train_adata.n_obs > n_samples:
        train_indices = np.random.choice(train_adata.n_obs, n_samples, replace=False)
        train_sample = train_adata[train_indices]
    else:
        train_sample = train_adata
        
    if test_adata.n_obs > n_samples:
        test_indices = np.random.choice(test_adata.n_obs, n_samples, replace=False)
        test_sample = test_adata[test_indices]
    else:
        test_sample = test_adata
    
    # Convert to dense matrices
    if hasattr(train_sample.X, 'toarray'):
        train_X = train_sample.X.toarray()
        test_X = test_sample.X.toarray()
    else:
        train_X = train_sample.X
        test_X = test_sample.X
    
    # Compute pairwise distances within each split
    train_distances = pairwise_distances(train_X, metric='cosine')
    test_distances = pairwise_distances(test_X, metric='cosine')
    
    # Compute distances between splits
    between_distances = pairwise_distances(train_X, test_X, metric='cosine')
    
    return {
        'train_within': train_distances[np.triu_indices_from(train_distances, k=1)],
        'test_within': test_distances[np.triu_indices_from(test_distances, k=1)],
        'between': between_distances.flatten()
    }

print("Computing distance matrices...")
distances = compute_distances_between_splits(train_adata, test_adata, n_samples=500)

# Visualize distance distributions
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Distance distributions
axes[0].hist(distances['train_within'], bins=50, alpha=0.7, label='Train within', density=True)
axes[0].hist(distances['test_within'], bins=50, alpha=0.7, label='Test within', density=True)
axes[0].hist(distances['between'], bins=50, alpha=0.7, label='Train-Test between', density=True)
axes[0].set_xlabel('Cosine Distance')
axes[0].set_ylabel('Density')
axes[0].set_title('Distance Distributions')
axes[0].legend()

# Box plot comparison
data_to_plot = [distances['train_within'], distances['test_within'], distances['between']]
labels = ['Train within', 'Test within', 'Train-Test between']
axes[1].boxplot(data_to_plot, labels=labels)
axes[1].set_ylabel('Cosine Distance')
axes[1].set_title('Distance Comparison')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Statistical comparison
print(f"Distance statistics:")
print(f"  Train within: {np.mean(distances['train_within']):.4f} ± {np.std(distances['train_within']):.4f}")
print(f"  Test within:  {np.mean(distances['test_within']):.4f} ± {np.std(distances['test_within']):.4f}")
print(f"  Between:      {np.mean(distances['between']):.4f} ± {np.std(distances['between']):.4f}")

# Test if between-split distances are significantly different
ks_stat, ks_pvalue = stats.ks_2samp(distances['train_within'], distances['between'])
print(f"\nKS test (train within vs between): stat={ks_stat:.4f}, p={ks_pvalue:.4f}")
if ks_pvalue < 0.05:
    print("*** SIGNIFICANT DIFFERENCE in distance distributions ***")


In [ ]:
# 5. Summary and Conclusions
print("\n=== SUMMARY AND CONCLUSIONS ===")

def generate_summary_report(statistical_results, pert_analysis, distances):
    """Generate a comprehensive summary report"""
    
    print("DISTRIBUTION ANALYSIS SUMMARY")
    print("=" * 50)
    
    # Gene expression differences
    print("\n1. GENE EXPRESSION DISTRIBUTION DIFFERENCES:")
    significant_diffs = []
    for metric, results in statistical_results.items():
        if results['ks_pvalue'] < 0.05:
            significant_diffs.append(metric)
            effect_size = "small" if abs(results['cohens_d']) < 0.5 else "medium" if abs(results['cohens_d']) < 0.8 else "large"
            print(f"   - {metric}: SIGNIFICANT (p={results['ks_pvalue']:.4f}, {effect_size} effect, d={results['cohens_d']:.3f})")
        else:
            print(f"   - {metric}: Not significant (p={results['ks_pvalue']:.4f})")
    
    # Perturbation analysis
    print(f"\n2. PERTURBATION PATTERN ANALYSIS:")
    print(f"   - Common perturbations: {len(pert_analysis['common_perts'])}")
    print(f"   - Test-only perturbations: {len(pert_analysis['test_only'])}")
    print(f"   - Train-only perturbations: {len(pert_analysis['train_only'])}")
    
    if len(pert_analysis['test_only']) > 0:
        print(f"   - *** OUT-OF-DISTRIBUTION TESTING: {len(pert_analysis['test_only'])} perturbations not seen in training ***")
    
    # Distance analysis
    print(f"\n3. DISTANCE-BASED ANALYSIS:")
    train_within_mean = np.mean(distances['train_within'])
    test_within_mean = np.mean(distances['test_within'])
    between_mean = np.mean(distances['between'])
    
    print(f"   - Train within-cluster distance: {train_within_mean:.4f}")
    print(f"   - Test within-cluster distance: {test_within_mean:.4f}")
    print(f"   - Train-Test between-cluster distance: {between_mean:.4f}")
    
    if between_mean > max(train_within_mean, test_within_mean) * 1.2:
        print(f"   - *** SIGNIFICANT SEPARATION: Train and test clusters are well-separated ***")
    
    # Overall conclusion
    print(f"\n4. OVERALL ASSESSMENT:")
    ood_indicators = []
    
    if len(significant_diffs) > 0:
        ood_indicators.append(f"Gene expression differences ({len(significant_diffs)} metrics)")
    
    if len(pert_analysis['test_only']) > 0:
        ood_indicators.append(f"Novel perturbations in test set ({len(pert_analysis['test_only'])} perturbations)")
    
    if between_mean > max(train_within_mean, test_within_mean) * 1.2:
        ood_indicators.append("Well-separated train/test clusters")
    
    if ood_indicators:
        print(f"   *** EVIDENCE OF OUT-OF-DISTRIBUTION TESTING:")
        for indicator in ood_indicators:
            print(f"      - {indicator}")
        print(f"\n   *** CONCLUSION: Fine-tuning performance issues likely due to OOD testing ***")
    else:
        print(f"   *** CONCLUSION: Limited evidence of OOD issues - other factors may be responsible ***")
    
    return {
        'significant_diffs': significant_diffs,
        'ood_indicators': ood_indicators,
        'is_ood': len(ood_indicators) > 0
    }

# Generate summary
summary = generate_summary_report(statistical_results, pert_analysis, distances)


In [ ]:
# Save results for further analysis
print("\n=== SAVING RESULTS ===")

# Create results directory
results_dir = Path("./distribution_analysis_results")
results_dir.mkdir(exist_ok=True)

# Save statistical results
import json
with open(results_dir / "statistical_results.json", "w") as f:
    # Convert numpy types to Python types for JSON serialization
    json_results = {}
    for metric, results in statistical_results.items():
        json_results[metric] = {k: float(v) if isinstance(v, (np.integer, np.floating)) else v 
                               for k, v in results.items()}
    json.dump(json_results, f, indent=2)

# Save perturbation analysis
pert_json = {
    'common_perts': list(pert_analysis['common_perts']),
    'train_only': list(pert_analysis['train_only']),
    'test_only': list(pert_analysis['test_only']),
    'train_single_count': pert_analysis['train_single_count'],
    'train_combo_count': pert_analysis['train_combo_count'],
    'test_single_count': pert_analysis['test_single_count'],
    'test_combo_count': pert_analysis['test_combo_count']
}
with open(results_dir / "perturbation_analysis.json", "w") as f:
    json.dump(pert_json, f, indent=2)

# Save summary
with open(results_dir / "summary.json", "w") as f:
    json.dump(summary, f, indent=2)

# Save plots
plt.savefig(results_dir / "distribution_analysis.png", dpi=300, bbox_inches='tight')

print(f"Results saved to {results_dir}/")
print(f"Files created:")
print(f"  - statistical_results.json")
print(f"  - perturbation_analysis.json") 
print(f"  - summary.json")
print(f"  - distribution_analysis.png")

print(f"\nAnalysis complete! Check the results directory for detailed outputs.")
